<a href="https://colab.research.google.com/github/lwazidladlaa/FirstRep/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import numpy as np # linear algebra
import re, os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf

import datetime
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

# Get the file details
directory = []
file = []
title = []
text = []
label = []

datapath = '/content/drive/MyDrive/zu/zu'

for dirname, _ , filenames in os.walk(datapath):
    #print('Directory: ', dirname)
    #print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)


DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)),
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['text','label'], axis=1)
print(df.shape)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(3313, 2)


,text,label
0,Ulwazi ngoHulumeni www.gov.za Izinkonzo Kuban...,NF-OBJ
1,Ulwazi ngoHulumeni www.gov.za Izinkonzo Kuban...,NF-OBJ
2,Izinkonzo Kubantu Izinkonzo ezinhlanganweni I...,NF-OBJ
3,Ulwazi ngoHulumeni www.gov.za Izinkonzo Kuban...,NF-OBJ
4,Ulwazi ngoHulumeni www.gov.za Izinkonzo Kuban...,NF-OBJ


In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()



In [ ]:
df['text'] = df['text'].map(preprocess)
df.head()


,text,label
0,ulwazi ngohulumeni www gov za izinkonzo kubant...,NF-OBJ
1,ulwazi ngohulumeni www gov za izinkonzo kubant...,NF-OBJ
2,izinkonzo kubantu izinkonzo ezinhlanganweni iz...,NF-OBJ
3,ulwazi ngohulumeni www gov za izinkonzo kubant...,NF-OBJ
4,ulwazi ngohulumeni www gov za izinkonzo kubant...,NF-OBJ


In [ ]:
df['label'] = '__label__' + df['label'].astype(str)
df.head(5)

,text,label
0,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ
1,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ
2,izinkonzo kubantu izinkonzo ezinhlanganweni iz...,__label__NF-OBJ
3,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ
4,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ


In [ ]:
df['label_text'] = df['label'] + " " + df['text']
df.head(3)

,text,label,label_text
0,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ,__label__NF-OBJ ulwazi ngohulumeni www gov za ...
1,ulwazi ngohulumeni www gov za izinkonzo kubant...,__label__NF-OBJ,__label__NF-OBJ ulwazi ngohulumeni www gov za ...
2,izinkonzo kubantu izinkonzo ezinhlanganweni iz...,__label__NF-OBJ,__label__NF-OBJ izinkonzo kubantu izinkonzo ez...


In [ ]:
df.dropna(inplace=True)
df.shape

(3313, 3)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)

In [ ]:
train.to_csv("isizulu.train", columns=["label_text"], index=False, header=False)
test.to_csv("isizulu.test", columns=["label_text"], index=False, header=False)

In [ ]:
!pip install fastText
import fasttext
import numpy as np

# Load your pre-trained model
model_isizulu = fasttext.load_model('/content/drive/MyDrive/zu/wiki.zu/wiki.zu.bin')

# Define a function to average word vectors for a document
def get_document_vector(doc, model):
    words = doc.split()  # Split the document into words
    # Initialize an array for the vectors of words
    word_vectors = [model.get_word_vector(word) for word in words if word in model.words]
    if len(word_vectors) > 0:
        # Average the word vectors
        document_vector = np.mean(word_vectors, axis=0)
        return document_vector
    else:
        return None

# Load your training and test data
train = pd.read_csv("isizulu.train", header=None, names=["label_text"])
test = pd.read_csv("isizulu.test", header=None, names=["label_text"])

# Create averaged vectors for your training and test data
train["label_text"] = train["label_text"].apply(lambda x: get_document_vector(x, model_isizulu))
test["label_text"] = test["label_text"].apply(lambda x: get_document_vector(x, model_isizulu))

# Remove rows with None values (documents that couldn't be represented as vectors)
train = train.dropna()
test = test.dropna()

In [ ]:
# Train a FastText model on the averaged vectors
model = fasttext.train_supervised(input="isizulu.train",
                                  dim=300,
                                  wordNgrams=2,
                                  minCount=3,
                                  epoch=100)

# Extract some training statistics
num_words = model.get_words()
num_labels = model.get_labels()

# Print the training statistics
print(f"Number of words: {num_words}")
print(f"Number of labels: {len(num_labels)}")

Number of words: ['noma', 'ukuthi', 'i', 'uma', 'isicelo', '1', 'kodwa', 'izinkonzo', 'kanye', 'u', 'kanti', 'phezulu', 'ukuba', 'ukuze', 'futhi', 'kakhulu', 'okumele', 'e', '2', 'lapho', 'kubantu', 'abantu', 'bona', 'indawo', 'kahle', 'le', 'imali', 'izinto', 'leli', 'ama', '</s>', 'kusho', 'engu', 'xhumana', 'umthetho', 'ngo', 'kufanele', 'nathi', 'yakho', 'ngoba', 'yendawo', 'ulwazi', 'khasi', 'zulu', 'lokhu', 'imephu', 'umuntu', 'kumele', 'isikhathi', 'emva', 'amafomu', '3', 'nge', 'ngesikhathi', 'khona', 'ngabe', 'izindleko', 'ngaphandle', 'za', 'okwenziwa', 'umsebenzi', 'of', 'izinyathelo', 'komsebenzi', 'afanele', 'amazinga', '4', 'nendlela', 'lokho', 'afrika', 'yokuxhumana', 'ne', 'no', 'agcwaliswe', 'ezinhlanganweni', 'osetshenziswayo', 'www', 'a', 'izincazelo', 'zilandelwe', 'ku', 'kwi', 'o', 'kukhona', 'the', 'z', 'mail', 'angaphandle', 'indlela', 'yini', 'izingane', 'ingane', 'yakhe', 'ngendlela', 'kuzo', 'wakhe', 'bamazwe', 'igama', 'lokuqala', 'njengoba', 'khetha', 'ka', 

In [ ]:
labels, probabilities = model.predict("zulu.1068.blockpuzzle (2) BONA ZULU – September 2006 BONA Block Puzzle Zithokozise uthuthukise ulwazi lwakho lwesiNgisi. Phendula ngokuyikho ulandele imicibisholo uzothola igama lomethuli wezemidlalo ezikweleni ezimnyama. No 59 WINA Amahampa okunakekela isikhumba abiza R8 000! AVE kulula ukugcina isikhumba somndeni wakho sinempilo siswakeme ngemikhiqizo eyikhwalithi yePink Lady yokunakekela isikhumba. Ngabe uyazi ngemikhiqizo yakho ongayethemba enjenge-petroleum jelly ne-glycerine? Manje ungazama i-aqueous cream ene-glycerine kanye ne-body lotion ene-cocoa butter, musk, floral ne-natural, kanye ne-camphor cream eyelaphayo. Bheka uchunge lwemikhiqizo yePink Lady esitolo sangakini - uzoyibona nemikhiqizo emisha ye-roll-on deodorants nensipho ugcwalise yonke imikhiqizo yePink Lady ekhaya lakho! UBONA unamahampa ayisishiyagalombili abiza R1 000 iyinye okumele awinwe. Kwihampa ngayinye kunemikhiqizo yePink Lady yomndeni wonke. Ukuze uwine Gcwalisa izikwele uthole igama elifihlekile kwezimnyama. Kwi-postcard, bhala impendulo, igama lakho, ikheli nenombolo yocingo otholakala kuyona ngezikhathi zasemini uyithumele kuleli kheli: BONA/Pink Lady Block Puzzle No 59, Box 32067, Mobeni, 4060.[z] Usuku lokuvala: 29 September 2006")
print(labels)
print(probabilities)

('__label__NF-SUB',)
[0.63747782]


In [ ]:
labels, probabilities = model.predict("Ukubika ubugebengu ngaphandle kokuzidalula igama Izincazelo Izinyathelo okumele zilandelwe Umthetho osetshenziswayo Amazinga afanele okwenziwa komsebenzi Izindleko Amafomu okumele agcwaliswe Indawo nendlela yokuxhumana Izincazelo Le nkonzo  ithokalala amahora angu-24 ngosuku kunoma ngubani umuntu ofuna ukubika ngociko ubugebengu noma isenzo sobugebengu ngokunikeza ulwazi olungasiza amaphoyisa ekuvimbeleni noma ekuphenyisiseni ngalobo bugebengu. Umshayeli angakhetha ukuthi angazidaluli igama lakhe. Khumbula ukuthi ungakashayela kule nombolo yolwazi lobugebengu  ethi: 08600 10111 Inombolo ephuthumayo yobugebengu i-Crime Emergencies ngu: 10111. Phezulu Izinyathelo okumele zilandelwe Shayela kwi-Crime Stop amahora angu-24 lonke usuku, kwinombolo ethi: 08600 10111 Oqaphile esikhungweni solwazi ngobugebengu, uzophendula ucingo kanye nokubhala lolo lwazi. Ungakhetha ukuthi ungazidaluli igama lakho. Oqapha ulwazi esikhungweni, uzokunikeza inombolo yekhodi eyimfihlo yeferense yangekusasa. Phezulu Umthetho osetshenziswayo South African Police Service Act, 1995 Phezulu Amazinga afanele okwenziwa komsebenzi Uzosizwa ngokushesha. Phezulu Izindleko Izingcingo zishajwa ngama-rate ezingcingo ezivamile kanti zishajwa ngama-rate amaselula .Phezulu Amafomu okumele agcwaliswe Awekho amafomu okumele agcwaliswe. Phezulu Indawo nendlela yokuxhumana")
print(labels)
print(probabilities)

('__label__NF-NEU',)
[0.95283473]


In [ ]:
labels, probabilities = model.predict("Siyakukhumbula ukucekelwa phansi kwamalungelo okwenzeka eminyakeni eyadlula;")

print(labels)
print(probabilities)

('__label__NF-SUB',)
[0.96099365]


In [ ]:
test_result=model.test("isizulu.test")
print("No. of samples,","precision,","recall")
print(test_result)

No. of samples, precision, recall
(994, 0.8058350100603622, 0.8058350100603622)


In [ ]:
model.save_model("classification_model_isizulu.bin")